In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
def getUrls(search):
    try:
        browser = webdriver.Chrome()
        browser.get('http://apps.webofknowledge.com/DIIDW_GeneralSearch_input.do?product=DIIDW')
        time.sleep(1)
        browser.find_element_by_name('value(input1)').send_keys(search)
        time.sleep(1)
        browser.find_element_by_xpath('//*[@id="searchCell1"]/span[1]/button').click()
        soup=BeautifulSoup(browser.page_source,'html.parser')
        urls=[]
        pattern= soup.find('a', attrs={'class': 'smallV110 snowplow-full-record'}).get('href').strip()
        for i in range(1,2):#获取前两页的内容
            for j in range(i*10-9,i*10+1): #每页有十条记录
                urls.append("https://apps.webofknowledge.com/"+pattern.replace('&page=1&doc=1','&page={}&doc={}'.format(i,j)))
        return urls
    except:
        print('没有找到相关专利')
        

In [4]:
def getImformation(urls):
    list=[]
    headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
        }
    if urls:
        for url in urls:
            data={}
            response=requests.get(url,headers=headers).text
            soup=BeautifulSoup(response)
            data['专利名称']=soup.find('td',attrs={'class':'FullRecTitle'}).text   
        
            td=soup.find_all('',attrs={'class':'fr_data_row'})
            data['专利号']=td[0].text.replace('Patent Number(s):','').strip()
            data['发明人']=td[1].text.replace('Inventor(s):','').strip()
            data['专利权人和代码']=td[2].text.replace('Patent Assignee Name(s) and Code(s): ','')
            data['Derwent 主入藏号:']=td[3].text.replace('Derwent Primary Accession Number: ','')
            data['摘要']=td[4].text.replace('Abstract:','').strip()
            data['国际专利分类']=td[6].text.replace('International Patent Classification:','').strip()
            data['德温特分类代码']=td[7].text.replace('Derwent Class Code(s):','').strip()
            data['德温特手工代码']=td[8].text.replace('Derwent Manual Code(s)::','').strip()
        
            list.append(data)
        df=pd.DataFrame(list)
        return df

In [5]:
def main():
    search=input('请输入查找关键字：')
    df=getImformation(getUrls(search))
    try:
        writer = pd.ExcelWriter('E:/3.xlsx')
        df.to_excel(writer,'Sheet1')
        writer.save()
    except:
        print("文件存储错误！")

In [6]:
main()

请输入查找关键字：5G
